## Load Dataset

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

In [3]:
# libgomp issue, must import n2 before torch
from n2 import HnswIndex

In [4]:
import sys

sys.path.insert(0, '..')

In [5]:
import os
home_dir = os.getenv('HOME')

https://dbs.uni-leipzig.de/research/projects/object_matching/benchmark_datasets_for_entity_resolution

In [6]:
from collections import defaultdict
import itertools

def Enumerator(start=0, initial=()):
    return defaultdict(itertools.count(start).__next__, initial)

In [7]:
import glob
import csv
from tqdm.auto import tqdm

id_enumerator = Enumerator()
row_dict = {}
left_id_set = set()
right_id_set = set()
rows_total = 2614 + 2294
clusters_total = 2224

with tqdm(total=rows_total) as pbar:
    with open(f'{home_dir}/Downloads/DBLP-ACM/DBLP2.csv', encoding="latin1") as f:
        for row in csv.DictReader(f):
            row['id'] = id_enumerator[row["id"]]
            row['source'] = 'acm'
            row_dict[row['id']] = row
            left_id_set.add(row['id'])
            pbar.update(1)
    
    with open(f'{home_dir}/Downloads/DBLP-ACM/ACM.csv', encoding="latin1") as f:
        for row in csv.DictReader(f):
            row['id'] = id_enumerator[row["id"]]
            row['source'] = 'dblp'
            row_dict[row['id']] = row
            right_id_set.add(row['id'])
            pbar.update(1)

In [8]:
true_pair_set = set()

with open(f'{home_dir}/Downloads/DBLP-ACM/DBLP-ACM_perfectMapping.csv') as f:
    for row in csv.DictReader(f):
        id_left = id_enumerator[row['idDBLP']]
        id_right = id_enumerator[row['idACM']]
        true_pair_set.add((id_left, id_right))

len(true_pair_set)

2224

In [9]:
from entity_embed.data_utils.utils import id_pairs_to_cluster_mapping_and_dict

cluster_mapping, cluster_dict = id_pairs_to_cluster_mapping_and_dict(true_pair_set)
len(cluster_mapping)

4448

In [10]:
len(cluster_dict)

2224

In [11]:
# TODO: deal with this difference
# from entity_embed.data_utils.utils import cluster_dict_to_id_pairs

# assert len(true_pair_set - cluster_dict_to_id_pairs(cluster_dict)) == 0

In [12]:
cluster_attr = 'cluster_id'
max_cluster_id = max(cluster_mapping.values())

for row_id, row in tqdm(row_dict.items()):
    try:
        row[cluster_attr] = cluster_mapping[row_id]
    except KeyError:
        row[cluster_attr] = max_cluster_id
        max_cluster_id += 1

In [13]:
[row_dict[row_id] for row_id in next(iter(true_pair_set))]

[{'id': 2046,
  'title': 'Versioning and Configuration Management in an Object-Oriented Data Model',
  'authors': 'Edward Sciore',
  'venue': 'VLDB J.',
  'year': '1994',
  'source': 'acm',
  'cluster_id': 2046},
 {'id': 3198,
  'title': 'Versioning and configuration management in an object-oriented data model',
  'authors': 'Edward Sciore',
  'venue': 'The VLDB Journal &mdash; The International Journal on Very Large Data Bases ',
  'year': '1994',
  'source': 'dblp',
  'cluster_id': 2046}]

## Preprocess

In [14]:
attr_list = ['title', 'authors', 'venue', 'year']

In [15]:
import unidecode
from entity_embed import default_tokenizer

def clean_str(s):
    s = unidecode.unidecode(s).lower().strip()
    s_tokens = itertools.islice((s_part[:30] for s_part in default_tokenizer(s)), 0, 30)
    return ' '.join(s_tokens)[:300]

for row in tqdm(row_dict.values()):
    for attr in attr_list:
        row[attr] = clean_str(row[attr])

## Init Data Module

In [16]:
import torch
import numpy as np

random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [17]:
alphabet = list('0123456789abcdefghijklmnopqrstuvwxyz!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ ')

In [18]:
attr_info_dict = {
    'title': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'authors': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'venue': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'year': {
        'field_type': "STRING",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    }
}

In [19]:
from entity_embed import build_row_numericalizer

row_numericalizer = build_row_numericalizer(attr_info_dict, row_dict=row_dict)
row_numericalizer.attr_info_dict

10:19:22 INFO:For attr='title', computing actual alphabet and max_str_len
10:19:22 INFO:For attr='title', using actual_max_str_len=18
10:19:22 INFO:For attr='authors', computing actual alphabet and max_str_len
10:19:22 INFO:For attr='authors', using actual_max_str_len=18
10:19:22 INFO:For attr='venue', computing actual alphabet and max_str_len
10:19:22 INFO:actual_max_str_len=13 must be pair to enable NN pooling. Updating to 14
10:19:22 INFO:For attr='venue', using actual_max_str_len=14
10:19:22 INFO:For attr='year', computing actual alphabet and max_str_len
10:19:22 INFO:For attr='year', using actual_max_str_len=4


{'title': NumericalizeInfo(field_type=<FieldType.MULTITOKEN: 'multitoken'>, tokenizer=<function default_tokenizer at 0x7f90470a94c0>, alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' '], max_str_len=18, vocab=None),
 'authors': NumericalizeInfo(field_type=<FieldType.MULTITOKEN: 'multitoken'>, tokenizer=<function default_tokenizer at 0x7f90470a94c0>, alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}'

In [20]:
from entity_embed import LinkageDataModule

train_cluster_len = 200
valid_cluster_len = 200
datamodule = LinkageDataModule(
    row_dict=row_dict,
    cluster_attr=cluster_attr,
    row_numericalizer=row_numericalizer,
    batch_size=20,
    row_batch_size=16,
    train_cluster_len=train_cluster_len,
    valid_cluster_len=valid_cluster_len,
    test_cluster_len=clusters_total - valid_cluster_len - train_cluster_len,
    only_plural_clusters=True,
    left_id_set=left_id_set,
    right_id_set=right_id_set,
    random_seed=random_seed
)

## Training

In [21]:
from entity_embed import LinkageEmbed

ann_k = 10
model = LinkageEmbed(
    datamodule,
    ann_k=ann_k,
    use_mask=True
)

In [22]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

max_epochs = 50
early_stop_callback = EarlyStopping(
   monitor='valid_recall_at_0.5',
   min_delta=0.00,
   patience=10,
   verbose=True,
   mode='max'
)
tb_log_dir = 'tb_logs'
tb_name = 'dblp-acm'
trainer = pl.Trainer(
    gpus=1,
    max_epochs=max_epochs,
    check_val_every_n_epoch=1,
    callbacks=[early_stop_callback],
    logger=TensorBoardLogger(tb_log_dir, name=tb_name),
)

10:19:22 INFO:GPU available: True, used: True
10:19:22 INFO:TPU available: None, using: 0 TPU cores
10:19:22 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [23]:
trainer.fit(model, datamodule)

10:19:22 INFO:Train pair count: 200
10:19:22 INFO:Valid pair count: 200
10:19:22 INFO:Test pair count: 1826
10:19:24 INFO:
  | Name        | Type       | Params
-------------------------------------------
0 | blocker_net | BlockerNet | 2.1 M 
1 | losser      | SupConLoss | 0     
-------------------------------------------
2.1 M     Trainable params
0         Non-trainable params
2.1 M     Total params


1

In [24]:
model.blocker_net.get_signature_weights()

{'title': 0.2776387333869934,
 'authors': 0.28044408559799194,
 'venue': 0.21557261049747467,
 'year': 0.2263445258140564}

## Testing

In [25]:
trainer.test(ckpt_path='best')

10:19:49 INFO:Train pair count: 200
10:19:49 INFO:Valid pair count: 200
10:19:49 INFO:Test pair count: 1826



--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_f1_at_0.3': 0.1425555208007507,
 'test_f1_at_0.5': 0.17821007869397332,
 'test_f1_at_0.7': 0.8080852954242558,
 'test_f1_at_0.9': 0.9667491068974993,
 'test_pair_entity_ratio_at_0.3': 6.508906549739654,
 'test_pair_entity_ratio_at_0.5': 5.10660454919156,
 'test_pair_entity_ratio_at_0.7': 0.7336256508632502,
 'test_pair_entity_ratio_at_0.9': 0.4971224993148808,
 'test_precision_at_0.3': 0.07675466296155951,
 'test_precision_at_0.5': 0.09783192014596974,
 'test_precision_at_0.7': 0.6794919686215913,
 'test_precision_at_0.9': 0.9696802646085998,
 'test_recall_at_0.3': 0.9989041095890411,
 'test_recall_at_0.5': 0.9989041095890411,
 'test_recall_at_0.7': 0.9967123287671232,
 'test_recall_at_0.9': 0.9638356164383561}
--------------------------------------------------------------------------------


[{'test_precision_at_0.3': 0.07675466296155951,
  'test_recall_at_0.3': 0.9989041095890411,
  'test_f1_at_0.3': 0.1425555208007507,
  'test_pair_entity_ratio_at_0.3': 6.508906549739654,
  'test_precision_at_0.5': 0.09783192014596974,
  'test_recall_at_0.5': 0.9989041095890411,
  'test_f1_at_0.5': 0.17821007869397332,
  'test_pair_entity_ratio_at_0.5': 5.10660454919156,
  'test_precision_at_0.7': 0.6794919686215913,
  'test_recall_at_0.7': 0.9967123287671232,
  'test_f1_at_0.7': 0.8080852954242558,
  'test_pair_entity_ratio_at_0.7': 0.7336256508632502,
  'test_precision_at_0.9': 0.9696802646085998,
  'test_recall_at_0.9': 0.9638356164383561,
  'test_f1_at_0.9': 0.9667491068974993,
  'test_pair_entity_ratio_at_0.9': 0.4971224993148808}]

## Testing manually 

In [26]:
# Only call this if test above wasn't run
# datamodule.setup(stage='test')

In [27]:
test_row_dict = datamodule.test_row_dict
test_left_vector_dict, test_right_vector_dict = model.predict(
    row_dict=test_row_dict,
    left_id_set=left_id_set,
    right_id_set=right_id_set,
    batch_size=16
)

In [28]:
embedding_size = model.blocker_net.embedding_size
test_true_pair_set = datamodule.test_true_pair_set

In [29]:
assert (len(test_left_vector_dict) + len(test_right_vector_dict)) == len(test_row_dict)

In [30]:
%%time

from entity_embed import ANNLinkageIndex

ann_index = ANNLinkageIndex(embedding_size=embedding_size)
ann_index.insert_vector_dict(left_vector_dict=test_left_vector_dict, right_vector_dict=test_right_vector_dict)
ann_index.build()

CPU times: user 1.24 s, sys: 18.2 ms, total: 1.25 s
Wall time: 192 ms


In [31]:
%%time

sim_threshold = 0.5
found_pair_set = ann_index.search_pairs(
    k=ann_k,
    sim_threshold=sim_threshold,
    left_vector_dict=test_left_vector_dict,
    right_vector_dict=test_right_vector_dict,
)

CPU times: user 1.21 s, sys: 0 ns, total: 1.21 s
Wall time: 153 ms


In [32]:
from entity_embed.evaluation import pair_entity_ratio

pair_entity_ratio(len(found_pair_set), len(test_row_dict))

5.107152644560154

In [33]:
from entity_embed.evaluation import precision_and_recall

precision_and_recall(found_pair_set, test_true_pair_set)

(0.09782142090577377, 0.9989041095890411)

In [34]:
false_positives = list(found_pair_set - test_true_pair_set)
len(false_positives)

16813

In [35]:
false_negatives = list(test_true_pair_set - found_pair_set)
len(false_negatives)

2

In [36]:
cos_similarity = lambda a, b: np.dot(a, b)

In [37]:
for (id_left, id_right) in false_negatives[:10]:
    display(
        (
            cos_similarity(test_left_vector_dict[id_left], test_right_vector_dict[id_right]),
            row_dict[id_left], row_dict[id_right]
        )
    )

(0.40427583,
 {'id': 1091,
  'title': 'introduction ( special issue on multimedia databases )',
  'authors': 'm . tamer ozsu , stavros christodoulakis',
  'venue': 'vldb j .',
  'year': '1998',
  'source': 'acm',
  'cluster_id': 1091},
 {'id': 3784,
  'title': 'introduction',
  'authors': 'm . tamer &# 214 ; zsu , stavros christodoulakis',
  'venue': 'the vldb journal & mdash ; the international journal on very large data bases',
  'year': '1998',
  'source': 'dblp',
  'cluster_id': 1091})

(-0.034067087,
 {'id': 2,
  'title': 'incremental maintenance for non - distributive aggregate functions',
  'authors': 'themistoklis palpanas , richard sidle , hamid pirahesh , roberta cochrane',
  'venue': 'vldb',
  'year': '2002',
  'source': 'acm',
  'cluster_id': 2615},
 {'id': 4022,
  'title': 'indexing and querying xml data for regular path expressions',
  'authors': 'quanzhong li , bongki moon',
  'venue': 'very large data bases',
  'year': '2001',
  'source': 'dblp',
  'cluster_id': 2615})